In [44]:
import cv2

video_path = "data/session5_center/video.avi"
json_gtruth_path = 'data/session5_center_gtruth/system_dubska_optimal_calib.json'

In [45]:
def play_video(video_path):
    video = cv2.VideoCapture(video_path)
    
    while video.isOpened():
        
        pos_x = 500
        pos_y = 600
        ret, frame = video.read()
        if ret:
        
            cv2.imshow('Video', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break
        
    video.release()
    cv2.destroyAllWindows()

In [46]:
#play_video(video_path)

In [47]:
import pickle

with open('data/session5_center/gt_data.pkl', 'rb') as pkl_file:
    speed_data = pickle.load(pkl_file, encoding='latin1')

In [48]:
import json

with open(json_gtruth_path, 'r') as json_file:
    position_data = json.load(json_file)

cars_data = position_data['cars']

In [55]:
cap = cv2.VideoCapture(video_path)


frame_number = 0
i = 0
j = 0


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_list = cars_data[i]['frames']
    car_id = cars_data[i]['id']
    
    if frame_list[j] == frame_number:
        pos_x = int(cars_data[i]['posX'][j])
        pos_y = int(cars_data[i]['posY'][j])
        
        img = cv2.circle(frame, (pos_x, pos_y), 20, (0,0,255), -1)
        img = cv2.putText(img, f"Car number {car_id}", (pos_x, pos_y), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0))
        j+=1
        
        if j == len(frame_list):
            i += 1
            j = 0
    
    else:
        img = frame
    

    
    cv2.imshow('frame', img)
        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
    frame_number += 1
    

    
cap.release()
cv2.destroyAllWindows()

In [50]:
cars_data[0]['id']

0